## James Cage
#### April 11, 2019

The intent of this notebook is to take the file "world_countries.json" and extract a database of country names and IDs

In [54]:
import csv
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library


In [55]:
# download countries geojson file
!wget --quiet https://ibm.box.com/shared/static/cto2qv7nx6yq19logfcissyy4euo8lho.json -O world_countries.json
    
print('GeoJSON file downloaded!')

GeoJSON file downloaded!


In [56]:
# Read the file into a Python string

with open('world_countries.json', "r") as file1:
    FileContent = file1.read()

In [57]:
# The following strings identify where the country names and ids 
# begin and end.

start_name = 'name'
end_name = '\"'
start_id = '\"id'
end_id = '\"' 


def mine(in_string, out_string):
    if len(in_string) < 20:
        return out_string
    s = in_string.find(start_name) + 7
    e = in_string.find(end_name, s)
    out_string += in_string[s:e]+","
    
    s = in_string.find(start_id, e) + 6
    e = in_string.find(end_id, s)
    out_string += in_string[s:e] + "\n"
    
    return mine(in_string[e:], out_string)


In [58]:
# Read in the json file, parse it, and save it as a CSV file

namesDB = "Country,ID\n"   # Define the file header

with open('world_names.csv', "w") as file2:
    file2.write(mine(FileContent, namesDB))
    
# Now read the CSV file back in and create a pandas dataframe
# The "round trip" proves the file is good.

df_names = pd.read_csv('world_names.csv')

## Fixing the Immigration Data
Open the immigration data and make all country names compatible with the json file (as stored in df_names)

In [59]:
# df_can = pd.read_excel('https://ibm.box.com/shared/static/lw190pt9zpy5bd1ptyg2aw15awomz9pu.xlsx',
#                      sheet_name='Canada by Citizenship',
#                      skiprows=range(20),
#                      skipfooter=2)

df_can = pd.read_excel('https://github.com/JamesDCage/31-31-Canada-01/blob/master/Canada.xlsx?raw=true',
                     sheet_name='Canada by Citizenship',
                     skiprows=range(20),
                     skipfooter=2)

df_can.head(5)

,Type,Coverage,OdName,AREA,AreaName,REG,RegName,DEV,DevName,1980,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Immigrants,Foreigners,Afghanistan,935,Asia,5501,Southern Asia,902,Developing regions,16,...,2978,3436,3009,2652,2111,1746,1758,2203,2635,2004
1,Immigrants,Foreigners,Albania,908,Europe,925,Southern Europe,901,Developed regions,1,...,1450,1223,856,702,560,716,561,539,620,603
2,Immigrants,Foreigners,Algeria,903,Africa,912,Northern Africa,902,Developing regions,80,...,3616,3626,4807,3623,4005,5393,4752,4325,3774,4331
3,Immigrants,Foreigners,American Samoa,909,Oceania,957,Polynesia,902,Developing regions,0,...,0,0,1,0,0,0,0,0,0,0
4,Immigrants,Foreigners,Andorra,908,Europe,925,Southern Europe,901,Developed regions,0,...,0,0,1,1,0,0,0,0,1,1


How many country names in df_can are not in our json file? What are those countries?
Let's find out using Python sets.

In [60]:
missing = list(set(df_can.OdName) - set(df_names.Country))
len(missing)

49

In [61]:
# 49 missing countries!! Can I just ignore these rows?
df_can_missing = df_can[df_can['OdName'].isin(missing)]
df_can_missing.sort_values(by=[2013], ascending=False).head(17)


,Type,Coverage,OdName,AREA,AreaName,REG,RegName,DEV,DevName,1980,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
81,Immigrants,Foreigners,Iran (Islamic Republic of),935,Asia,5501,Southern Asia,902,Developing regions,1172,...,6348,5837,7480,6974,6475,6580,7477,7479,7534,11291
183,Immigrants,Foreigners,United Kingdom of Great Britain and Northern I...,908,Europe,924,Northern Europe,901,Developed regions,22045,...,7533,7258,7140,8216,8979,8876,8724,6204,6195,5827
140,Immigrants,Foreigners,Republic of Korea,935,Asia,906,Eastern Asia,902,Developing regions,1011,...,5352,5832,6215,5920,7294,5874,5537,4588,5316,4509
143,Immigrants,Foreigners,Russian Federation,908,Europe,923,Eastern Europe,901,Developed regions,0,...,3989,3972,3117,2983,2690,2931,2288,1963,2079,2466
190,Immigrants,Foreigners,Viet Nam,935,Asia,920,South-Eastern Asia,902,Developing regions,1191,...,1816,1852,3153,2574,1784,2171,1942,1723,1731,2112
141,Immigrants,Foreigners,Republic of Moldova,908,Europe,923,Eastern Europe,901,Developed regions,0,...,628,655,803,1099,1127,1535,1988,1367,1416,1231
111,Immigrants,Foreigners,Mauritius,903,Africa,910,Eastern Africa,902,Developing regions,306,...,702,696,512,507,714,872,1455,1146,799,1203
43,Immigrants,Foreigners,Côte d'Ivoire,903,Africa,914,Western Africa,902,Developing regions,14,...,256,294,431,619,646,668,1066,636,1024,1170
189,Immigrants,Foreigners,Venezuela (Bolivarian Republic of),904,Latin America and the Caribbean,931,South America,902,Developing regions,103,...,1224,1211,1192,1335,1239,1353,998,1452,1373,1022
169,Immigrants,Foreigners,Syrian Arab Republic,935,Asia,922,Western Asia,902,Developing regions,315,...,1116,1458,1145,1056,919,917,1039,1005,650,1009


In [62]:
# Ignoring ALL of the rows would be a bad idea. 
# Iran alone contributes ~ 4% to the immigration total for that year.
# But many contribute less than 0.05% of the total in 2013. 
# Let's ignore those. Now create a list of countries we WON'T ignore.

threshold =  0.0005 * df_can[2013].sum()

df_can_missing[df_can_missing[2013] > threshold].OdName

# I'm okay with ignoring any row that contributes less than 0.1% of immigration 

# df_can['Total'] = df_can.iloc[:, 9:43].sum(axis=1)

37        China, Hong Kong Special Administrative Region
43                                         Côte d'Ivoire
70                                               Grenada
81                            Iran (Islamic Republic of)
111                                            Mauritius
140                                    Republic of Korea
141                                  Republic of Moldova
143                                   Russian Federation
146                                          Saint Lucia
147                     Saint Vincent and the Grenadines
153                                               Serbia
156                                            Singapore
163                                   State of Palestine
169                                 Syrian Arab Republic
183    United Kingdom of Great Britain and Northern I...
189                   Venezuela (Bolivarian Republic of)
190                                             Viet Nam
Name: OdName, dtype: object

Of the above, the following were not in the json file. They're all fairly low in the list.

China, Hong Kong Special Administrative Region
Grenada
Mauritius
Saint Lucia
Saint Vincent and the Grenadines
Singapore
State of Palestine


I saved the updated spreadsheet as 'Canada2.xlsx'. Let's read that in now.

In [63]:
df_can = pd.read_excel('https://github.com/JamesDCage/31-31-Canada-01/blob/master/Canada2.xlsx?raw=true',
                     sheet_name='Canada by Citizenship',
                     skiprows=range(20),
                     skipfooter=2)

df_can.head(5)

,Type,Coverage,OdName,AREA,AreaName,REG,RegName,DEV,DevName,1980,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Immigrants,Foreigners,Afghanistan,935,Asia,5501,Southern Asia,902,Developing regions,16,...,2978,3436,3009,2652,2111,1746,1758,2203,2635,2004
1,Immigrants,Foreigners,Albania,908,Europe,925,Southern Europe,901,Developed regions,1,...,1450,1223,856,702,560,716,561,539,620,603
2,Immigrants,Foreigners,Algeria,903,Africa,912,Northern Africa,902,Developing regions,80,...,3616,3626,4807,3623,4005,5393,4752,4325,3774,4331
3,Immigrants,Foreigners,American Samoa,909,Oceania,957,Polynesia,902,Developing regions,0,...,0,0,1,0,0,0,0,0,0,0
4,Immigrants,Foreigners,Andorra,908,Europe,925,Southern Europe,901,Developed regions,0,...,0,0,1,1,0,0,0,0,1,1


### Fixing World Population Data

Let's import world population data from the World Bank, and make sure that country names are compatible. Since we've (mostly) made the names in df_can compatible with the geojson file, then we can use df_names to check for & enforce compatibility with df_names.

In [68]:
df_world_pop = pd.read_excel('http://api.worldbank.org/v2/en/indicator/SP.POP.TOTL?downloadformat=excel',
                     sheet_name='Data',
                     skiprows=range(3))
df_world_pop.head(2)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,...,101453.0,101669.0,102053.0,102577.0,103187.0,103795.0,104341.0,104822.0,105264.0,NaN
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996351.0,9166764.0,9345868.0,9533954.0,9731361.0,9938414.0,...,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0,32758020.0,33736494.0,34656032.0,35530081.0,NaN


In [69]:
# Generate a list of columns we don't want, and then drop them.

bad_columns = list(df_world_pop.loc[:, 'Indicator Name':'1979'].columns) + \
              list(df_world_pop.loc[:, '2014':'2018'].columns)

df_world_pop.drop(bad_columns, axis=1, inplace=True)

# Rename the first column to match the column in df_can

df_world_pop.rename(columns={'Country Name':'Country'}, inplace=True)

df_world_pop.head()

,Country,Country Code,1980,1981,1982,1983,1984,1985,1986,1987,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Aruba,ABW,60096.0,60567.0,61345.0,62201.0,62836.0,63026.0,62644.0,61833.0,...,98737.0,100031.0,100832.0,101220.0,101353.0,101453.0,101669.0,102053.0,102577.0,103187.0
1,Afghanistan,AFG,13248370.0,13053954.0,12749645.0,12389269.0,12047115.0,11783050.0,11601041.0,11502761.0,...,24118979.0,25070798.0,25893450.0,26616792.0,27294031.0,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0
2,Angola,AGO,8929900.0,9244507.0,9582156.0,9931562.0,10277321.0,10609042.0,10921037.0,11218268.0,...,18865716.0,19552542.0,20262399.0,20997687.0,21759420.0,22549547.0,23369131.0,24218565.0,25096150.0,25998340.0
3,Albania,ALB,2671997.0,2726056.0,2784278.0,2843960.0,2904429.0,2964762.0,3022635.0,3083605.0,...,3026939.0,3011487.0,2992547.0,2970017.0,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0
4,Andorra,AND,36067.0,37500.0,39114.0,40867.0,42706.0,44600.0,46517.0,48455.0,...,76244.0,78867.0,80991.0,82683.0,83861.0,84462.0,84449.0,83751.0,82431.0,80788.0


In [70]:
# How many countries in df_names do not have a matching entry in df_world_pop? And are they countries of interest?

missing = list(set(df_names.Country) - set(df_world_pop['Country']))
df_can_missing = df_can[df_can['OdName'].isin(missing)]
df_can_missing.sort_values(by=[2013], ascending=False).head()

,Type,Coverage,OdName,AREA,AreaName,REG,RegName,DEV,DevName,1980,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
81,Immigrants,Foreigners,Iran,935,Asia,5501,Southern Asia,902,Developing regions,1172,...,6348,5837,7480,6974,6475,6580,7477,7479,7534,11291
185,Immigrants,Foreigners,United States of America,905,Northern America,905,Northern America,901,Developed regions,9378,...,6990,8394,9613,9463,10190,8995,8142,7676,7891,8501
140,Immigrants,Foreigners,South Korea,935,Asia,906,Eastern Asia,902,Developing regions,1011,...,5352,5832,6215,5920,7294,5874,5537,4588,5316,4509
55,Immigrants,Foreigners,Egypt,903,Africa,912,Northern Africa,902,Developing regions,612,...,2393,2496,2190,2356,3347,3496,5982,4663,5555,4165
143,Immigrants,Foreigners,Russia,908,Europe,923,Eastern Europe,901,Developed regions,0,...,3989,3972,3117,2983,2690,2931,2288,1963,2079,2466


I see 17 countries, including big ones like Iran, the US, South Korea, etc. 

But here's a stroke of luck. The World Pop data has a "Country Code" that appears to be the same as the "ID" in df_names. If they are the same, then we can rename countries in df_world_pop to be compatible with the geojson file ... and we made the immigration data (more or less) compatible with that file above.

To merge two dataframes in Pandas, it appears the column headings must be the same. Let's make that happen.

In [71]:
df_world_pop.rename(columns={'Country Code':'ID'}, inplace=True)
df_world_pop.head(2)

,Country,ID,1980,1981,1982,1983,1984,1985,1986,1987,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Aruba,ABW,60096.0,60567.0,61345.0,62201.0,62836.0,63026.0,62644.0,61833.0,...,98737.0,100031.0,100832.0,101220.0,101353.0,101453.0,101669.0,102053.0,102577.0,103187.0
1,Afghanistan,AFG,13248370.0,13053954.0,12749645.0,12389269.0,12047115.0,11783050.0,11601041.0,11502761.0,...,24118979.0,25070798.0,25893450.0,26616792.0,27294031.0,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0


In [72]:
# Let's merge the frames and compare country names.

pd.merge(df_world_pop, df_names, on='ID')[['Country_x', 'Country_y']].head()

,Country_x,Country_y
0,Afghanistan,Afghanistan
1,Angola,Angola
2,Albania,Albania
3,United Arab Emirates,United Arab Emirates
4,Argentina,Argentina


In [73]:
# So the ID is a way to match country names across databases. 
# Now let's change the country names in df_world_pop to the equivalents in df_names

# First, create sets of IDs in each dataframe
df_names_ids = set(df_names.ID)
df_world_pop_ids = set(df_world_pop.ID)

# Our source is df_names, so iterate through those IDs
for id in set(df_names.ID):
    # If the ID is in df_world_pop, then find the country name in df_names
    # and use it to set the country name in df_world_pop
    if id in df_world_pop_ids:
        # Find the index of the row in df_names with this ID
        a = df_names.index[df_names['ID'] == id] 
        # Get the country name from this row
        country = df_names.at[int(list(a)[0]),'Country']
        # Find the index of the row in df_world_pop with this ID
        b = df_world_pop.index[df_world_pop['ID'] == id]
        # Set the country in df_world_pop equal to the name from df_names
        df_world_pop.at[int(list(b)[0]),'Country'] = country

In [74]:
# Let's check again - How many countries in df_names do not have a matching entry in df_world_pop? And are they countries of interest?

missing = list(set(df_names.Country) - set(df_world_pop['Country']))
df_can_missing = df_can[df_can['OdName'].isin(missing)]
df_can_missing.sort_values(by=[2013], ascending=False).head()

,Type,Coverage,OdName,AREA,AreaName,REG,RegName,DEV,DevName,1980,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
191,Immigrants,Foreigners,Western Sahara,903,Africa,912,Northern Africa,902,Developing regions,0,...,0,0,1,0,0,0,0,0,0,0


We're down to 1 row, with 0 immigration in 2013. Now let's write the world_pop data to a file, so we can use it elsewhere.

In [75]:
csv_data = df_world_pop.to_csv(index=False)
with open('world_population.csv', "w") as file2:
    file2.write(csv_data)

### And we're done! 

We'll see world_population.csv and Canada2.xlsx in the next notebook, "Normalized_Canadian_Immigration_V0.ipynb"

In [76]:
# 04-16-19